# Intel® Extension for Scikit-learn Lasso Regression for YearPredictionMSD dataset

In [1]:
from time import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data/YearPredictionMSD.txt', header=None)

In [3]:
x = data.iloc[:, 1:].to_numpy(dtype=np.float32)
y = data.iloc[:, 0].to_numpy(dtype=np.float32)
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle=False,
                                                    train_size=463715,
                                                    test_size=51630)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

In [5]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((463810, 90), (51535, 90), (463810,), (51535,))

Normalize the data

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
scaler_x = MinMaxScaler()
scaler_y = StandardScaler()

In [8]:
scaler_x.fit(x_train)
x_train = scaler_x.transform(x_train)
x_test = scaler_x.transform(x_test)

In [9]:
scaler_y.fit(y_train.reshape(-1, 1))
y_train = scaler_y.transform(y_train.reshape(-1, 1)).ravel()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

Intel Extension for Scikit-learn (previously known as daal4py) contains drop-in replacement functionality for the stock scikit-learn package. You can take advantage of the performance optimizations of Intel Extension for Scikit-learn by adding just two lines of code before the usual scikit-learn imports:

In [10]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Intel(R) Extension for Scikit-learn patching affects performance of specific Scikit-learn functionality. Refer to the [list of supported algorithms and parameters](https://intel.github.io/scikit-learn-intelex/algorithms.html) for details. In cases when unsupported parameters are used, the package fallbacks into original Scikit-learn. If the patching does not cover your scenarios, [submit an issue on GitHub](https://github.com/intel/scikit-learn-intelex/issues).

Training of the Lasso algorithm with Intel(R) Extension for Scikit-learn for YearPredictionMSD dataset

In [11]:
from sklearn.linear_model import Lasso

In [12]:
params = {
    "alpha": 0.5,    
    "fit_intercept": False,
    "random_state": 0,
    "copy_X": False
}

In [13]:
start = time()
model = Lasso(**params).fit(x_train, y_train)
f"Intel(R) extension for Scikit-learn time: {(time() - start):.2f} s"

'Intel(R) extension for Scikit-learn time: 0.04 s'

Predict and get a result of the Lasso algorithm with Intel(R) Extension for Scikit-learn

In [14]:
y_predict = model.predict(x_test)
mse_metric = metrics.mean_squared_error(y_test, y_predict)
mse_metric

1.0171133

In order to cancel optimizations, we use *unpatch_sklearn* and reimport the class Lasso

In [15]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

Training of the Lasso algorithm with original scikit-learn library for YearPredictionMSD dataset

In [16]:
from sklearn.linear_model import Lasso

In [17]:
start = time()
model = Lasso(**params).fit(x_train, y_train)
f"Original Scikit-learn time: {(time() - start):.2f} s"

'Original Scikit-learn time: 0.53 s'

Predict and get a result of the Lasso algorithm with original Scikit-learn

In [18]:
y_predict = model.predict(x_test)
mse_metric = metrics.mean_squared_error(y_test, y_predict)
mse_metric

1.0171133

With scikit-learn-intelex patching you can:

- Use your scikit-learn code for training and prediction with minimal changes (a couple of lines of code);
- Fast execution training and prediction of scikit-learn models;
- Get the same quality;
- Get speedup more than **13** times.